In [32]:
!pip install datasets==3.6.0

In [ ]:
from google.colab import files
uploaded = files.upload()
reviews = pd.read_csv("imdb.csv").sample(500)
reviews.head()

In [ ]:
import sys
!{sys.executable} -m pip install peft
import peft
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer

from peft import get_peft_model, LoraConfig, TaskType

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
reviews = reviews.rename(columns={"review": "text", "sentiment": "label"})
reviews['label'] = LabelEncoder().fit_transform(reviews['label'])
reviews.head()

,text,label
842,Outstanding film of 1943 with Paul Lukas givin...,1
8721,"Lame, ridiculous and absurd. My 6 year old son...",0
6911,This movie is a waste of time and money. Throu...,0
4819,I'm disappointed at the lack of posts on this ...,1
2562,"even though this movie is quite old, no matter...",1


To help with this task, we're going to use the [Datasets library](https://huggingface.co/docs/datasets/en/index), which allows for working with various types of datasets, from text, to audio, to images.

First, convert the reviews DataFrame into a dataset object by using the [from_pandas function](https://huggingface.co/docs/datasets/en/index).

In [ ]:
dataset = Dataset.from_pandas(reviews)

Once converted, we can perform a train/test split using a method of the Dataset object.

Peform an 80/20 train/test split using the [train_test_split method](https://huggingface.co/docs/datasets/en/index). Save the result back to the same Dataset object.

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

Now, extract the training and test portion into separate Datasets. Name these new datasets train_dataset and test_dataset, respectively.

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

We're going to be working with a DistilBERT model, which means that we'll need to tokenize our input in the way that DistilBERT expects. For this, we can use the [DistilBertTokenizerFast](https://huggingface.co/docs/transformers/en/model_doc/distilbert?usage=AutoModel#transformers.DistilBertTokenizerFast) tokenizer.

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Then we'll apply our tokenizer to the training and test datsets using the map method.

In [ ]:
train_dataset = train_dataset.map(lambda df: tokenizer(df['text'], padding="max_length", truncation=True), batched=True)
test_dataset = test_dataset.map(lambda df: tokenizer(df['text'], padding="max_length", truncation=True), batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Finally, we'll set the format to torch so that we're working with [PyTorch](https://pytorch.org/) tensors and only extract the columns that we need.

In [ ]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Now, let's load in the pretrained DistilBERT model.

### Part 1: Fine-tuning All Parameters

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, we need to set up a [Trainer](https://huggingface.co/docs/transformers/en/main_classes/trainer) for our model.

First, create a [TraininingArguments](https://huggingface.co/docs/transformers/v4.52.3/en/main_classes/trainer#transformers.TrainingArguments) object. Set num_train_epochs to 5, weight_decay to 0.01, and report_to = 'none'

In [ ]:
training_arg = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none",
)

Finally, create a Trainer object using the model, the Training Arguments that you created, and with the train_dataset equal to train_dataset.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_arg,
    train_dataset=train_dataset)

Now, use the [train method](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Trainer.train) on your Trainer object.

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=0.20243572998046874, metrics={'train_runtime': 95.9422, 'train_samples_per_second': 20.846, 'train_steps_per_second': 2.606, 'total_flos': 264934797312000.0, 'train_loss': 0.20243572998046874, 'epoch': 5.0})

Once the model has been fit, use the [predict method](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Trainer.predict) of the Trainer object to generate a set of predictions on the test_dataset.

In [ ]:
predictions = trainer.predict(test_dataset)

In [ ]:
labels = predictions.label_ids
predicted_labels = np.argmax(predictions.predictions, axis=1)

Finally, look the confusion matrix and classification report for these predictions.

In [ ]:
cm = confusion_matrix(labels, predicted_labels)
cr = classification_report(labels, predicted_labels)
print(cm)
print(cr)

[[50  5]
 [ 7 38]]
              precision    recall  f1-score   support

           0       0.88      0.91      0.89        55
           1       0.88      0.84      0.86        45

    accuracy                           0.88       100
   macro avg       0.88      0.88      0.88       100
weighted avg       0.88      0.88      0.88       100



### Part 2: Training Only a Subset of the Parameters

Let's first reload the pretrained distilbert model.

Then, we'll make none of the parameters trainable by setting the `requires_grad` attribute to False.

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

for param in model.distilbert.parameters():
    param.requires_grad = False

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Then, we'll go back and make the last 2 layers trainable.

In [ ]:
for i in [4, 5]:
    for param in model.distilbert.transformer.layer[i].parameters():
        param.requires_grad = True

We'll now set up TrainingArguments and a Trainer as before to train the model.

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    weight_decay=0.01,
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=0.298244140625, metrics={'train_runtime': 54.725, 'train_samples_per_second': 36.546, 'train_steps_per_second': 4.568, 'total_flos': 264934797312000.0, 'train_loss': 0.298244140625, 'epoch': 5.0})

How do the results of fine-tuning only the last two layers compare to fine-tuning all layers? How does the training time compare?

In [ ]:
predictions = trainer.predict(test_dataset)
true_labels = predictions.label_ids
predicted_labels = np.argmax(predictions.predictions, axis=1)
print(confusion_matrix(true_labels, predicted_labels))
print(classification_report(true_labels, predicted_labels))

[[48  7]
 [ 6 39]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88        55
           1       0.85      0.87      0.86        45

    accuracy                           0.87       100
   macro avg       0.87      0.87      0.87       100
weighted avg       0.87      0.87      0.87       100



The results of the fine-tuning only the last two layers is comparable to the fine-tuning all layers and gave slightly lower accuracy as of 0.87 vs 0.88 from the fine-tuning all layers. The training time of the fine-tuning only the last two layers at 55s is shorter than those of the fine-tuning all layers (1m 35s).

### Part 3: Parameter-Efficient Fine-Tuning

Now, let's see how we can use the [peft library](https://huggingface.co/docs/peft/en/index) to more efficiently fine-tune our model.

First, we'll re-initalize the model.

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create a [LoraConfig object](https://huggingface.co/docs/peft/en/index) were you set the Lora attention dimension to 8, the lora_alpha to 16, the lora_dropout to 0.1, the target_modules to "q_lin" and "v_lin" (these are the "query" and "value" projections), and set the task_type to TaskType.SEQ_CLS.

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"],
    task_type=TaskType.SEQ_CLS)

Now, use the [get_peft_model function](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.get_peft_model) to create the Lora model pass in the distilbert model and the LoraConfig object that you created.

In [ ]:
lora_model = get_peft_model(model, lora_config)

How many trainable parameters does the resulting model have? Hint: you can use the [print_trainable_parameters function](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.PeftModel.print_trainable_parameters).

In [ ]:
from peft import print_trainable_parameters
print_trainable_parameters(lora_model)

ImportError: cannot import name 'print_trainable_parameters' from 'peft' (/usr/local/lib/python3.11/dist-packages/peft/__init__.py)

We'll again set up the same TrainingArguments and create our Trainer object to grain the model.

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    weight_decay=0.01,
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=0.622558349609375, metrics={'train_runtime': 73.3694, 'train_samples_per_second': 27.259, 'train_steps_per_second': 3.407, 'total_flos': 269478813696000.0, 'train_loss': 0.622558349609375, 'epoch': 5.0})

How does the performance of the lora model compare to the previous two? How does the training time compare?

In [ ]:
predictions = trainer.predict(test_dataset)
true_labels = predictions.label_ids
predicted_labels = np.argmax(predictions.predictions, axis=1)
print(confusion_matrix(true_labels, predicted_labels))
print(classification_report(true_labels, predicted_labels))

[[49  6]
 [13 32]]
              precision    recall  f1-score   support

           0       0.79      0.89      0.84        55
           1       0.84      0.71      0.77        45

    accuracy                           0.81       100
   macro avg       0.82      0.80      0.80       100
weighted avg       0.81      0.81      0.81       100



The lora model preformance is lower than the previous two models with longer training time compared to the fine-tuning only the last two layers but shorter than those of the fine-tuning all layers.